# Setup

In [1]:
import os
import json
import lancedb
# import dsutils
import numpy as np
import pyarrow as pa
from glob import glob
from tqdm import tqdm
from docarray.typing import NdArray
from typing import Optional
from docarray import BaseDoc, DocList
from FlagEmbedding import BGEM3FlagModel
from docarray.index import HnswDocumentIndex
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [2]:
class Document(BaseDoc):
    _id: str
    timestamp: int
    query: str
    title: str
    press: str
    summary: Optional[str] = None
    content: str
    url: str
    origin: str
    embedding: Optional[NdArray] = None

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "summary" in "Document" shadows an attribute in parent "BaseDoc"
  warnings.warn(


## MongoDB

In [3]:
client = MongoClient()
hostname = 'mongo.stockhelper-mongodb.store'
username = 'root'
password = 'financial'
client = MongoClient(hostname, username=username, password=password)
db = client['financial']
news = db.news.find({})

In [4]:
def convert_objectid(doc):
    if "_id" in doc:
        doc["_id"] = str(doc["_id"])
        
    if 'timestamp' in doc:
        doc['timestamp'] = int(doc['timestamp'].timestamp())

    return doc

In [5]:
news = list(news)
news = [convert_objectid(doc) for doc in news]

In [6]:
print(news[0])
print(len(news))

{'_id': '66668bfa598cbc832134cd28', 'timestamp': 1718027415, 'query': '삼성전자', 'title': 'SK하이닉스 목표주가 벽 2번 부쉈는데… 삼성전자는 뒷걸음질만', 'press': '조선비즈', 'summary': None, 'content': '삼성전자 주가가 오를만하면 다시 뒷걸음질 치면서 증권사가 제시하는 목표주가와 차이를 좁히지 못하고 있다. 인공지능(AI) 열풍에 힘입어 SK하이닉스가 올해 들어 두 차례 목표주가를 돌파한 것과 대조적이다.삼성전자 주식은 10일 오후 1시 30분 유가증권시장에서 7만5800원에 거래됐다. 전 거래일보다 주가가 1.94%(1500원) 하락했다. 같은 시각 SK하이닉스 주가는 전 거래일보다 0.24%(500원) 올랐다. 이날 국내 주식시장은 미국 5월 고용지표가 시장 예상을 웃돌면서 전반적으로 부진한데, 기업별로 어느 정도 온도 차가 있는 상황이다.이날 만의 일이 아니다. 삼성전자와 SK하이닉스 주가 흐름은 올해 들어 엇갈려 왔다. AI 핵심 반도체인 고대역폭메모리(HBM)가 희비를 갈랐다. SK하이닉스는 AI 반도체 시장의 90% 이상을 점유하고 있는 미국 엔비디아에 4세대 HBM(HBM3)에 이어 5세대 HBM(HBM3E)을 납품하면서 주가가 탄력을 받았다. 삼성전자는 엔비디아에 제품 공급을 위해 HBM3E ‘퀄 테스트(최종 신뢰성 평가)’를 진행 중이지만, 아직 최종 승인이 나오지 않았다.SK하이닉스 주가는 올해 초 13만9700원에서 이날까지 48.9%(6만8300원) 오르며 증권사 기대치를 경신했다. 연초 목표주가 평균 16만4000원을 지난 3월 넘어섰고, 5월 초 목표주가 평균 18만2000원도 같은 달 13일부터 웃돌았다. SK하이닉스에 대한 목표주가를 제시한 국내외 증권사·투자은행(IB)은 현재 SK하이닉스의 목표주가로 평균 23만3800원을 제시하고 있다. 시티그룹은 SK하이닉스의 목표주가를 최고치인 31만원까지 올려놓은 상태다.삼성전자는 반대로 

## LanceDB

In [7]:
uri = "/workspace/008_PseudoLab/server/vectorstore"
db = lancedb.connect(uri)
# async_db = await lancedb.connect_async(uri)

In [8]:
schema = pa.schema([
    pa.field('id', pa.string()),
    pa.field('timestamp', pa.int64()),
    pa.field('query', pa.string()),
    pa.field('title', pa.string()),
    pa.field('press', pa.string()),
    pa.field('summary', pa.string()),
    pa.field('content', pa.string()),
    pa.field('url', pa.string()),
    pa.field('origin', pa.string()),
    pa.field("embedding", pa.list_(pa.float32(), list_size=1024))
])

table = db.create_table("news", schema=schema, mode="overwrite")

In [9]:
table = db.open_table('news')
# async_tbl = async_db.open_table('news')

In [10]:
embedding_function = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cuda')

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 393216.00it/s]
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
from tqdm.auto import tqdm

docs = []

for data in tqdm(news):
    tmp = 'timestamp: ' + str(datetime.fromtimestamp(data['timestamp'])) + '\n' \
        + 'title: ' + data['title'] + '\n' \
            + 'content: ' + data['  `content']
    embedding = embedding_function.encode(tmp, return_dense=True, return_sparse=False)
    data['embedding'] = embedding['dense_vecs']
    docs.append(Document(**data))

docs = DocList[Document](docs)
docs = [dict(d) for d in docs]
table.add(docs)

100%|██████████| 13236/13236 [06:37<00:00, 33.33it/s]


In [ ]:
# async_db = await lancedb.connect_async(uri)

## LanceDB Test

In [ ]:
table = db.open_table('news')
table.count_rows()
# async_tbl.count_rows()

In [ ]:
# table.search("2024-06-10").limit(10).to_list()
table.head()

In [ ]:
embedding_function = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cpu')

In [ ]:
from datetime import datetime

query = '삼성전자의 주식과 관련된 뉴스를 알려줘'
query = embedding_function.encode(query)['dense_vecs']
start_time = '2024-06-08'
start_stamp = datetime.strptime(start_time, "%Y-%m-%d").timestamp()

end_time = '2024-06-10'
end_stamp = datetime.strptime(end_time, "%Y-%m-%d").timestamp()
print(end_stamp)
end_stamp += 3600
print(end_stamp)
docs = table.search(query).where(f"(timestamp >= {int(start_stamp)}) AND (timestamp < {int(end_stamp)})", prefilter=True).limit(5).to_list()
# docs = table.search(query).limit(5).to_list()
# docs = async_tbl.search(query).limit(8).to_list()

In [ ]:
end_time = '2024-06-11'
end_stamp = datetime.strptime(end_time, "%Y-%m-%d").timestamp()
print(end_stamp)

In [ ]:
[(str(datetime.fromtimestamp(doc['timestamp'])), doc['title']) for doc in docs]

In [ ]:
sorted([(str(datetime.fromtimestamp(doc['timestamp'])), doc['title']) for doc in docs], key=lambda x : x[0])

In [ ]:
query = '최근 삼성전자의 신제품에 대해 알려줘'
query = embedding_function.encode(query)['dense_vecs']
docs = table.search(query).limit(5).where(f"timestamp > {int(date)}", prefilter=True).to_list()

In [ ]:
sorted([(str(datetime.fromtimestamp(doc['timestamp'])), doc['title']) for doc in docs], key=lambda x : x[0])

In [ ]:
a = "asidhfioas"

a[:12034]